In [2]:
import requests
import time
import hmac
import hashlib
import json
import pandas as pd
from datetime import datetime

# -----------------------------
# CONFIGURACIÓN DE LA CUENTA
# -----------------------------
API_KEY    = "bg_afdcb9221ad98efb3b0b7bdd4c236338"
API_SECRET = "0c4214cbfccfb648f841b43ca5d68531c8fb44b75ab271fdd222da9a74ee413f"
API_PASSPHRASE = "Cryptobitget86"

BASE_URL = "https://api.bitget.com"

INITIAL_CAPITAL = 837  # Capital inicial en USDT
PRODUCT_TYPE = "USDT-FUTURES"

# -----------------------------
# FUNCIONES DE FIRMA Y PETICIÓN
# -----------------------------
def get_timestamp():
    return str(int(time.time() * 1000))

def sign_request(timestamp, method, request_path, body=""):
    message = f"{timestamp}{method.upper()}{request_path}{body}"
    h = hmac.new(API_SECRET.encode('utf-8'), message.encode('utf-8'), hashlib.sha256)
    return h.hexdigest()

def make_request(method, endpoint, params=None):
    url = f"{BASE_URL}{endpoint}"
    body = json.dumps(params) if params else ""
    timestamp = get_timestamp()
    sign = sign_request(timestamp, method, endpoint, body)
    
    headers = {
        "ACCESS-KEY": API_KEY,
        "ACCESS-SIGN": sign,
        "ACCESS-TIMESTAMP": timestamp,
        "ACCESS-PASSPHRASE": API_PASSPHRASE,
        "Content-Type": "application/json"
    }

    if method.upper() == "GET":
        r = requests.get(url, headers=headers, params=params)
    else:
        r = requests.post(url, headers=headers, data=body)
    
    return r.json()

# -----------------------------
# OBTENER HISTORIAL DE TRADES

def get_account_bills(symbol=None, limit=100, start_time=None, end_time=None, page_no=1):
    endpoint = "/api/mix/v2/account/bills"
    url = BASE_URL + endpoint

    params = {
        "productType": PRODUCT_TYPE,
        "limit": limit,
        "pageNo": page_no
    }
    if symbol:
        params["symbol"] = symbol
    if start_time:
        params["startTime"] = int(start_time.timestamp() * 1000)
    if end_time:
        params["endTime"] = int(end_time.timestamp() * 1000)

    timestamp = get_timestamp()
    # Para firmar, usamos la ruta sin query string
    sign = sign_request(timestamp, "GET", endpoint, "")

    headers = {
        "ACCESS-KEY": API_KEY,
        "ACCESS-SIGN": sign,
        "ACCESS-TIMESTAMP": timestamp,
        "ACCESS-PASSPHRASE": API_PASSPHRASE,
        "Content-Type": "application/json"
    }

    r = requests.get(url, headers=headers, params=params)
    
    try:
        response = r.json()
    except json.JSONDecodeError:
        print("❌ Error decodificando JSON:", r.text)
        return []

    if response["code"] != "00000":
        print("Error fetching bills:", response)
        return []

    return response["data"]["bills"] if "data" in response and "bills" in response["data"] else []



# -----------------------------
# OBTENER BALANCE TOTAL USDT
# -----------------------------
def get_total_balance():
    endpoint = f"/api/v2/mix/account/accounts?productType={PRODUCT_TYPE}"
    response = make_request("GET", endpoint)
    if response["code"] != "00000":
        print("Error fetching account list:", response)
        return 0
    return float(response["data"][0]["usdtEquity"])

# -----------------------------
# CALCULAR RENTABILIDAD Y WIN RATIO
# -----------------------------
def calculate_profitability(bills):
    if not bills:
        print("⚠️ No se encontraron trades")
        return

    df = pd.DataFrame(bills)
    df['realizedPnl'] = df['realizedPnl'].astype(float)
    
    # Resumen por símbolo
    summary = []
    for symbol, group in df.groupby("symbol"):
        total_pnl = group["realizedPnl"].sum()
        total_trades = len(group)
        positive_trades = (group["realizedPnl"] > 0).sum()
        summary.append([symbol, total_trades, positive_trades, total_pnl])

    df_summary = pd.DataFrame(summary, columns=["Symbol", "Total Trades", "Positive Trades", "Profit (USDT)"])
    print("\n📊 Rentabilidad por símbolo:")
    print(df_summary)

    # Resumen global
    total_trades = len(df)
    total_positive = (df["realizedPnl"] > 0).sum()
    total_profit = df["realizedPnl"].sum()
    total_balance = get_total_balance()
    total_return_percent = (total_balance - INITIAL_CAPITAL) / INITIAL_CAPITAL * 100
    win_ratio_percent = (total_positive / total_trades) * 100 if total_trades > 0 else 0

    print(f"\n💰 Balance total USDT: {total_balance:.2f}")
    print(f"📈 Rentabilidad total: {total_return_percent:.2f}%")
    print(f"🏆 Win Ratio total : {win_ratio_percent:.2f}% ({total_positive}/{total_trades} trades positivos)")

# -----------------------------
# SCRIPT PRINCIPAL
# -----------------------------
if __name__ == "__main__":
    since_date = "2025-09-16"
    start_time = datetime.strptime(since_date, "%Y-%m-%d")

    bills = get_account_bills(start_time=start_time)
    calculate_profitability(bills)


❌ Error decodificando JSON: 404 not found.

⚠️ No se encontraron trades
